In [2]:
!pip install numpy 
!pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [7]:
!pip install json
!pip install csv


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [2]:
import json
import csv
import os
from langdetect import detect, LangDetectException
å
for x in range(2, 31):
    # Define the path to your directory containing the JSON files
    directory_path = f'/Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 {x}/1/{x}'

    # Initialize a set to keep track of unique tweet texts
    unique_texts = set()

    # Open the combined CSV file for writing
    with open(f'combined_english_unique_tweets_withdate_{x}.csv', 'w', newline='', encoding='utf-8') as combined_csv_file:
        # Initialize CSV writer and write the headers
        csv_writer = csv.writer(combined_csv_file)
        csv_headers = ['Tweet ID', 'Author ID', 'Author Username', 'Text', 'Retweet Count', 'Reply Count', 'Like Count', 'Quote Count', 'Impression Count', 'Date Created']  # Add 'Date Created' header
        csv_writer.writerow(csv_headers)
        
        # Loop through each subdirectory and file based on your structure
        for x in range(24):
            subdirectory_path = os.path.join(directory_path, str(x))
            if os.path.exists(subdirectory_path):
                for i in range(60):
                    input_file_path = os.path.join(subdirectory_path, f'{i}.json')
                    if os.path.isfile(input_file_path):
                        with open(input_file_path, 'r', encoding='utf-8') as json_file:
                            for line in json_file:
                                try:
                                    data = json.loads(line)
                                    tweets = data['includes']['tweets']
                                    users = data['includes']['users']
                                    date_created = data['data']['created_at']  # Extract date created
                                    
                                    for tweet in tweets:
                                        text = tweet['text'].lower()  # Case-insensitive matching
                                        if ('bitcoin' in text or 'btc' in text) and (text not in unique_texts):
                                            # Add the text to the set of seen texts to track uniqueness
                                            unique_texts.add(text)
                                           
                                            # Proceed  if the tweet is in English
                                            try:
                                                if detect(text) == 'en':
                                                    tweet_id = tweet['id']
                                                    author_id = tweet['author_id']
                                                    author_username = next((user['username'] for user in users if user['id'] == author_id), '')
                                                    retweet_count = tweet['public_metrics']['retweet_count']
                                                    reply_count = tweet['public_metrics']['reply_count']
                                                    like_count = tweet['public_metrics']['like_count']
                                                    quote_count = tweet['public_metrics']['quote_count']
                                                    impression_count = tweet['public_metrics'].get('impression_count', 0)  # Default to 0 if missing
                                                    csv_writer.writerow([tweet_id, author_id, author_username, text, retweet_count, reply_count, like_count, quote_count, impression_count, date_created])  # Write date created
                                            except LangDetectException:
                                                continue
                                except KeyError:
                                    continue

    print("Combined CSV file created successfully!")


Combined CSV file created successfully!
Combined CSV file created successfully!
Combined CSV file created successfully!
Combined CSV file created successfully!
Combined CSV file created successfully!
Combined CSV file created successfully!


In [6]:
import pandas as pd

df = pd.read_csv("tweets_with_sentiment_2.csv", engine="python")

df

,Tweet ID,Author ID,Author Username,Text,Retweet Count,Reply Count,Like Count,Quote Count,Impression Count,Sentiment Score
0,1609700113236254720,1.477111e+18,Coinzonestats,the seven day moving average of crypto exchang...,0.0,2.0,2.0,0.0,0.0,0.718855
1,1609701242455965696,1.527865e+18,sunshin99308875,rt @bscnews: when #core mainnet? 🌐 @huobigloba...,213.0,0.0,0.0,0.0,0.0,1.511833
2,1609625667695435776,1.323762e+18,BSCNews,when #core mainnet? 🌐 @huobiglobal is ready! t...,213.0,95.0,841.0,8.0,0.0,1.723703
3,1609701364107542528,2.558961e+08,shannon_aman,@bitcoin4freedom thank u for all u do! hope u ...,0.0,0.0,0.0,0.0,0.0,1.984804
4,1609701682870423556,4.305902e+08,hamada02019772,"@amoreyba wow, did you get his last btc update...",0.0,0.0,0.0,0.0,0.0,1.684770
...,...,...,...,...,...,...,...,...,...,...
5435,1610062289037824008,5.434832e+08,natbrunell,the internet was information for the people.\n...,6.0,5.0,20.0,0.0,0.0,1.492941
5436,1609520605933768711,1.463919e+18,CryptoInsightsX,🏅coin of the day🏅\n\n@dollarmoonio $dmoon\n\n🔥...,52.0,27.0,212.0,1.0,0.0,0.692381
5437,1610058108927184896,9.678593e+17,IIICapital,a bitcoin private key is 32 bytes.\n\na human ...,11.0,9.0,45.0,1.0,0.0,0.714729
5438,1609284246493097986,1.344080e+18,TYonClubhouse,calling all lawyers &amp; friends of lawyers!\...,6.0,2.0,11.0,0.0,0.0,1.837904


In [1]:
import bz2
import os
import shutil
def unzip_bz2_files(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Folder does not exist: {folder_path}")
        return

    # Use os.scandir to get all .bz2 files in the specified folder
    try:
        with os.scandir(folder_path) as entries:
            for entry in entries:
                if entry.is_file() and entry.name.endswith('.bz2'):
                    file_path = entry.path
                    # Destination file path (without the .bz2 extension)
                    dest_file_path = os.path.splitext(file_path)[0]
                    
                    # Decompress the .bz2 file
                    try:
                        with bz2.open(file_path, 'rb') as f_in:
                            with open(dest_file_path, 'wb') as f_out:
                                # Directly copy the decompressed data
                                shutil.copyfileobj(f_in, f_out)
                        print(f"Successfully decompressed: {file_path}")
                    except Exception as e:
                        print(f"Failed to decompress {file_path}: {e}")
    except Exception as e:
        print(f"Error scanning folder {folder_path}: {e}")

if __name__ == "__main__":
    base_path = "/Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items"
    for date in range(25, 31):  # Assuming you want to loop through specific days
        for hour in range(24):  # Hours 0-23
            for minute in range(60):  # Minutes 0-59
                # Corrected folder path construction
                folder_path = f"{base_path}/2023 {date}/{hour}/{date}/{minute}/"
                if os.path.exists(folder_path):
                    print(folder_path)
                    unzip_bz2_files(folder_path)
                else:
                    print(f"Path does not exist: {folder_path}")


Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/0/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/1/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/2/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/3/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/4/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/5/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/6/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/7/
Path does not exist: /Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 25/0/25/8/
Path does not exist: /Volumes/T7/archiveteam-t

/Volumes/T7/archiveteam-twitter-stream-2023-01/New Folder With Items/2023 18/1/18/0